In [1]:
from dataclasses import dataclass
from typing import Iterable, List, Dict, Optional

In [2]:
@dataclass
class Knapsack:
    weight: Iterable[int]
    profit: Iterable[int]
    capacity: int

    def __len__(self):
        assert len(self.weight) == len(self.profit)
        return len(self.weight)


@dataclass
class KnapsackSolution:
    profit: int
    items: List[int]


@dataclass
class ContinuousKnapsackSolution:
    profit: float
    items_x: Dict[int, float]

    def integer(self) -> bool:
        return all(x.is_integer() for x in self.items_x.values())
    
    def items(self) -> List[int]:
        assert self.integer()
        return [i for i, val in self.items_x.items() if val == 1.0]

In [3]:
class BBNode:
    kp: Knapsack
    fixed: List[int]
    free: List[int]
    parent_sol: Optional[ContinuousKnapsackSolution]
    base_profit: int
    capacity: int

    def __init__(self, kp: Knapsack, fixed: List[int], free: List[int], parent_sol: Optional[ContinuousKnapsackSolution] = None):
        self.kp = kp
        self.fixed = fixed
        self.free = free
        self.parent_sol = parent_sol
        self.base_profit = sum(self.kp.profit[i] for i in self.fixed)
        self.capacity = self.kp.capacity - sum(self.kp.weight[i] for i in self.fixed)

    def solve(self) -> Optional[ContinuousKnapsackSolution]:
        excluded = set(range(len(self.kp))) - set(self.fixed) - set(self.free)
        print(f"\tFixed: {', '.join(map(str, self.fixed))}")
        print(f"\tExcluded: {', '.join(map(str, excluded))}")
        print(f"\tFree: {', '.join(map(str, self.free))}")
        print(f"\tResidual capacity: {self.capacity}")

        if self.capacity < 0:
            return None
        
        if self.parent_sol is not None:
            if len(self.free) > 0:
                branching_item = min(self.free) - 1
            else:
                branching_item = len(self.kp) - 1

            if (x := self.parent_sol.items_x[branching_item]) == 1.0:
                print(f"\tBranching item {branching_item} was selected in parent node (x = {x}): shortcutting.")
                return self.parent_sol

        residual = self.capacity
        items_x = {i: 0.0 for i in range(len(self.kp))}

        for i in self.free:
            if self.kp.weight[i] < residual:
                items_x[i] = 1.0
                residual -= self.kp.weight[i]
            else:
                items_x[i] = residual / self.kp.weight[i]
                break

            if residual == 0.0:
                break

        profit = sum(x * self.kp.profit[i] for i, x in items_x.items()) + self.base_profit
        for i in self.fixed:
            items_x[i] = 1.0

        print(f"\tOptimal sol: {', '.join(f'{i}: {val}' for i, val in items_x.items())}")

        return ContinuousKnapsackSolution(profit=profit, items_x=items_x)

class BBTree:
    kp: Knapsack
    nodes: List[BBNode]
    best_solution: Optional[KnapsackSolution]

    def __init__(self, kp: Knapsack):
        self.kp = kp
        self.nodes = [self.__root_node()]
        self.best_solution = None

    def __root_node(self) -> BBNode:
        return BBNode(kp=self.kp, fixed=list(), free=list(range(len(self.kp))), parent_sol=None)
    
    def __update_best(self, sol: ContinuousKnapsackSolution) -> None:
        assert sol.integer()

        if self.best_solution is None or self.best_solution.profit < sol.profit:
            self.best_solution = KnapsackSolution(profit=sol.profit, items=sol.items())

    def __insert_node_with_fixed(self, item: int, current_node: BBNode, current_sol: ContinuousKnapsackSolution) -> None:
        fixed = current_node.fixed + [item]
        free = current_node.free.copy()
        free.remove(item)
        self.nodes.append(BBNode(kp=self.kp, fixed=fixed, free=free, parent_sol=current_sol))

    def __insert_node_with_excluded(self, item: int, current_node: BBNode, current_sol: ContinuousKnapsackSolution) -> None:
        fixed = current_node.fixed.copy()
        free = current_node.free.copy()
        free.remove(item)
        self.nodes.append(BBNode(kp=self.kp, fixed=fixed, free=free, parent_sol=current_sol))
    
    def solve(self) -> Optional[KnapsackSolution]:
        node_n = 0

        while len(self.nodes) > 0:
            print(f"Exploring node {node_n}...")
            
            node = self.nodes.pop()
            node_n += 1
            sol = node.solve()

            # If no free item is left, the resulting KP is either
            # infeasible or completely determined (integer solution).
            assert len(node.free) > 0 or (sol is None or sol.integer())

            if sol is None:
                # Infeasible Node
                print(f"Node {node_n} infeasible.")
                continue

            if sol.integer():
                # Leaf Node
                print(f"Node {node_n} is a leaf node. Sol profit = {sol.profit}.")
                self.__update_best(sol)
                continue

            if self.best_solution is not None and sol.profit < self.best_solution.profit:
                # Sub-optimal node: prune
                print(f"Node {node_n} suboptimal. Dual bound = {sol.profit} < {self.best_solution.profit} = primal bound.")
                continue

            min_item = min(node.free)

            print(f"Node {node_n} has a fractional optimum. Branching on item {min_item}.")
            self.__insert_node_with_excluded(item=min_item, current_node=node, current_sol=sol)
            self.__insert_node_with_fixed(item=min_item, current_node=node, current_sol=sol)

        print(f"Explored {node_n} nodes.")
        if self.best_solution is not None:
            print(f"Optimal solution with profit {self.best_solution.profit}.")
        else:
            print(f"Problem infeasible.")

        return self.best_solution

In [4]:
kp = Knapsack(weight=(2,2,2,4,4,6), profit=(5,4,3,5,4,5), capacity=12)
bb = BBTree(kp=kp)

In [5]:
sol = bb.solve()

Exploring node 0...
	Fixed: 
	Excluded: 
	Free: 0, 1, 2, 3, 4, 5
	Residual capacity: 12
	Optimal sol: 0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 0.5, 5: 0.0
Node 1 has a fractional optimum. Branching on item 0.
Exploring node 1...
	Fixed: 0
	Excluded: 
	Free: 1, 2, 3, 4, 5
	Residual capacity: 10
	Branching item 0 was selected in parent node (x = 1.0): shortcutting.
Node 2 has a fractional optimum. Branching on item 1.
Exploring node 2...
	Fixed: 0, 1
	Excluded: 
	Free: 2, 3, 4, 5
	Residual capacity: 8
	Branching item 1 was selected in parent node (x = 1.0): shortcutting.
Node 3 has a fractional optimum. Branching on item 2.
Exploring node 3...
	Fixed: 0, 1, 2
	Excluded: 
	Free: 3, 4, 5
	Residual capacity: 6
	Branching item 2 was selected in parent node (x = 1.0): shortcutting.
Node 4 has a fractional optimum. Branching on item 3.
Exploring node 4...
	Fixed: 0, 1, 2, 3
	Excluded: 
	Free: 4, 5
	Residual capacity: 2
	Branching item 3 was selected in parent node (x = 1.0): shortcutting.
Node 5 has